In [ ]:
// hacky way to set java path since macos breaks this
val userhome = System.getProperty("user.home")
val jniPath = s"${userhome}/software/z3/build"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
//set sys_paths to null so that java.library.path will be reevaluated next time it is needed
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);
// note: make sim link between bounder and Bounder due to macos case sensitivity before hand
val path = s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
//unzip -p jdiff.jar META-INF/MANIFEST.MF
interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._

In [ ]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.Action
import edu.colorado.plv.bounder.RunConfig
import edu.colorado.plv.bounder.PickleSpec
import edu.colorado.plv.bounder.lifestate.SpecSpace
import edu.colorado.plv.bounder.lifestate.SpecSignatures
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await
Driver.setZ3Path(s"${userhome}/software/z3/build")

var android_home_possible = List(s"${userhome}/Library/Android/sdk", s"${userhome}/Android/Sdk")
var android_home = android_home_possible.find(p => File(p).exists()).get             
BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> s"${userhome}/software/z3/build","ANDROID_HOME" -> android_home,"HOME" -> userhome))

Experimental setup: chose 3(TODO increase) open source apps and ran no model on all of them.  Chose 10 alarm dereferences from each app at random and wrote specs to prove.

- we excluded alarms where a callin method or callback simply may not be invoked with a null value (e.g. null = a.getLayoutInflator() -[]-> false)


In [ ]:
case object Experiments{
    //val specs = PickleSpec.mk(new SpecSpace(Set(SpecSignatures.Activity_getLayoutInflater_nonNull)))
    val configs = List(
    //sgtpuzzles
    read[RunConfig]("""{"apkPath":"${baseDir}/sgtpuzzles/app/build/outputs/apk/debug/app-debug.apk","outFolder":["${baseDirOut}/name.boyle.chris/name.boyle.chris.sgtpuzzles.databinding.ActivityHelpBinding"],"initialQuery":[{"t":"AllReceiversNonNull","className":"name.boyle.chris.sgtpuzzles.databinding.ActivityHelpBinding"}],"timeLimit":180}""")
    )//.map{c => c.copy(specSet = specs)}
    val outfolder = File(s"${userhome}/Desktop/specsRun")
    assert(outfolder.exists())
}

In [ ]:
Experiments.configs.zipWithIndex.map{
    case (c:RunConfig,a:Int) => 
        val fname = Experiments.outfolder / s"${a}.json"
        fname.overwrite(write(c))
}